In [ ]:
import clickhouse_connect
import pandas as pd

In [ ]:
# Connect to ClickHouse
client = clickhouse_connect.get_client(
    host='localhost', 
    port=8123,
    username='default',
    password='ClickHousePassword'
)

In [ ]:
create_mv_sql = """
CREATE MATERIALIZED VIEW mv_flights_by_minute (
    minute DateTime,
    total_flights UInt64,
    unique_aircraft UInt64,
    avg_altitude Float64,
    airport_counts AggregateFunction(uniq, String)
) ENGINE = AggregatingMergeTree()
ORDER BY minute
AS SELECT
    toStartOfMinute(timestamp) as minute,
    count() as total_flights,
    uniq(flight_id) as unique_aircraft,
    avg(altitude) as avg_altitude,
    uniqState(airport_code) as airport_counts
FROM adsb_messages
GROUP BY minute;
"""

client.command(create_mv_sql)

query_table_sql = """SELECT * FROM mv_flights_by_minute LIMIT 10"""

result = client.query(query_table_sql)

df = result.result_set

display(df)

In [ ]:
client.command("DROP TABLE  mv_flights_by_minute")